# Structural Break Detection – Baseline with Feature Extraction & Gradient Boosting

This notebook develops a **first strong baseline** for the CrunchDAO Structural Break challenge:

1. Load raw training data (variable-length sequences).
2. Extract **summary statistics features** around the break boundary.
3. Save the extracted features for reuse.
4. Train a **HistGradientBoostingClassifier** with cross-validation & hyperparameter tuning.
5. Evaluate on hold-out set, report **ROC-AUC**.
6. Save model, CV results, and prediction probabilities.



In [4]:
import pandas as pd
from pathlib import Path
from scipy.stats import kurtosis, skew
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.utils.class_weight import compute_sample_weight
import joblib

# --- Rich features: robust stats + higher moments + FFT + wavelets ---
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from numpy.fft import rfft
import pywt  # pip install PyWavelets


In [5]:
def _stats_robust(x):
    if x.size == 0:
        return {}
    q5, q10, q25, q50, q75, q90, q95 = np.percentile(x, [5, 10, 25, 50, 75, 90, 95])
    mad = np.median(np.abs(x - q50))  # Median Absolute Deviation
    return {
        "mean": np.mean(x), "std": np.std(x),
        "min": np.min(x),  "max": np.max(x),  "median": q50,
        "q5": q5, "q10": q10, "q25": q25, "q75": q75, "q90": q90, "q95": q95,
        "mad": mad, "skew": skew(x, bias=False) if x.size > 2 else 0.0, "kurt": kurtosis(x, fisher=True, bias=False) if x.size > 3 else 0.0,
        "rms": np.sqrt(np.mean(x**2)),  "ptp": np.ptp(x),  # peak-to-peak
    }

def _fft_energy_bands(x, n_bands=5):
    """Simple relative-band energies from power spectrum."""
    if x.size < 8:
        return {f"fft_band_{i}": 0.0 for i in range(n_bands)}
    spec = np.abs(rfft(x - np.mean(x)))**2
    spec = spec[1:]  # drop DC for energy ratios
    if spec.size == 0:
        return {f"fft_band_{i}": 0.0 for i in range(n_bands)}
    # split into equal bands
    bands = np.array_split(spec, n_bands)
    energies = np.array([b.sum() for b in bands], dtype=float)
    tot = energies.sum() + 1e-12
    return {f"fft_band_{i}": float(e/tot) for i, e in enumerate(energies)}

def _wavelet_energies(x, wavelet="db4", level=None):
    """Wavelet packet-ish: energy per scale from DWT coefficients."""
    if x.size < 8:
        return {}
    coeffs = pywt.wavedec(x - np.mean(x), wavelet=wavelet, level=level)
    energies = [np.sum(c**2) for c in coeffs]  # [cA_L, cD_L, ..., cD1]
    tot = np.sum(energies) + 1e-12
    out = {"wl_cA": float(energies[0]/tot)}
    for i, e in enumerate(energies[1:], start=1):
        out[f"wl_cD_{i}"] = float(e/tot)
    return out

def _segment_features(x):
    """Compose robust stats + spectral features for one segment."""
    f = {}
    f.update(_stats_robust(x))
    f.update(_fft_energy_bands(x, n_bands=5))
    f.update(_wavelet_energies(x))
    return f

def extract_features_rich(X: pd.DataFrame) -> pd.DataFrame:
    """Per-id features using value and period columns."""
    feats = []
    print_count = 0
    for id_, g in X.groupby(level="id"):

        if print_count == 1_000:
            print('Progress report...extracting features from id: {}'.format(id_))
            print_count = 0
        print_count += 1

        v = g["value"].values.astype(float)
        pre = g.loc[g["period"] == 0, "value"].values.astype(float)
        post = g.loc[g["period"] == 1, "value"].values.astype(float)

        d = {"id": id_}
        # global
        d.update({f"g_{k}": v for k, v in _segment_features(v).items()})

        # pre/post
        d.update({f"pre_{k}": v for k, v in _segment_features(pre).items()})
        d.update({f"post_{k}": v for k, v in _segment_features(post).items()})

        # deltas (post - pre) for key stats
        for k in ["mean", "std", "median", "mad", "skew", "kurt", "rms"]:
            d[f"delta_{k}"] = (d.get(f"post_{k}", 0.0) - d.get(f"pre_{k}", 0.0))

        # counts & ratio
        d["len_total"] = int(v.size)
        d["n_pre"] = int(pre.size)
        d["n_post"] = int(post.size)
        d["ratio_post_pre"] = float(d["n_post"]/(d["n_pre"]+1e-6))
        feats.append(d)

    df = pd.DataFrame(feats).set_index("id")
    return df.replace([np.inf, -np.inf], 0.0).fillna(0.0)


In [7]:
RESOURCES_DIR = Path("../resources")
RESOURCES_DIR.mkdir(exist_ok=True)

def train_cv_tuned_model(X: pd.DataFrame, y: pd.Series):
    """
    Tune HGB hyperparameters with StratifiedKFold on ROC-AUC, then refit on all data.
    Returns (best_estimator, best_params, cv_results_df).
    """
    est = HistGradientBoostingClassifier(
        random_state=42,
        early_stopping=True,          # quick regularisation
        validation_fraction=0.1,
    )

    # A compact grid for ROC-AUC .... expand if time allows
    param_grid = {
        "learning_rate": [0.03, 0.06],
        "max_depth": [3, 5, 7],
        "max_iter": [200, 400],
        "min_samples_leaf": [10, 25, 100],
        "l2_regularization": [0.0, 1e-2, 1e-1],
        "max_bins": [255],            # default is strong; tune if you wish
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    sample_weight = compute_sample_weight("balanced", y) # handle possible class imbalance

    gs = GridSearchCV(
        estimator=est,
        param_grid=param_grid,
        scoring="roc_auc",
        cv=cv,
        n_jobs=-1,
        verbose=3,
        refit=True,   # refit on full data with best params automatically
    )

    gs.fit(X, y, sample_weight=sample_weight)
    best_est = gs.best_estimator_
    best_params = gs.best_params_
    cv_results = pd.DataFrame(gs.cv_results_).sort_values("rank_test_score")

    print("\n[CV] Best ROC-AUC (mean cv):", gs.best_score_)
    print("[CV] Best params:", best_params)
    return best_est, best_params, cv_results


def evaluate_on_holdout(model, X_test: pd.DataFrame, y_test: pd.Series):
    """Report holdout ROC-AUC and return probability DataFrame [P(y=0), P(y=1)]."""
    proba = model.predict_proba(X_test)
    auc = roc_auc_score(y_test, proba[:, 1])
    print("[Holdout] ROC-AUC:", auc)
    proba_df = pd.DataFrame(
        proba, index=X_test.index, columns=["P(y=0)", "P(y=1)"]
    )
    return auc, proba_df



In [8]:
# ==== end-to-end ====
# 0) Load data
from preprocess import load_data
X_raw, X_test_raw, y_train, y_test = load_data('../data')

# 1) extract features
FEATURE_DIR = Path("../data/feature/extraction")
FEATURE_DIR.mkdir(parents=True, exist_ok=True)



In [21]:
try:
    X_train_moments = pd.read_parquet(FEATURE_DIR / "X_train_moments.parquet")
    X_test_moments = pd.read_parquet(FEATURE_DIR / "X_test_moments.parquet")
    X_test_moments = X_test_moments[X_train_moments.columns]
except:
    X_train_moments = extract_features_rich(X_raw).fillna(0.0)
    X_test_moments  = extract_features_rich(X_test_raw).fillna(0.0)
    X_test_moments = X_test_moments[X_train_moments.columns]
    X_train_moments.to_parquet(FEATURE_DIR / "X_train_moments.parquet")
    X_test_moments.to_parquet(FEATURE_DIR / "X_test_moments.parquet")

In [37]:
def train_model(X_train, y_train, l2_regularization=0.0, max_leaf_nodes=3.1):
    model = HistGradientBoostingClassifier(random_state=42, l2_regularization=l2_regularization, max_leaf_nodes=max_leaf_nodes)
    model.fit(X_train, y_train)
    return model

def evaluate(model, X_val, y_val):
    y_pred_proba = model.predict_proba(X_val)[:,1]
    auc = roc_auc_score(y_val, y_pred_proba)
    print("Validation ROC AUC:", auc)
    return auc


In [43]:
auc = evaluate(train_model(X_train_moments, y_train, 0.0, 31), X_test_moments, y_test)
auc = evaluate(train_model(X_train_moments, y_train, 1e-2, 100), X_test_moments, y_test)
model = train_model(X_train_moments, y_train, 1e-2, 100)

Validation ROC AUC: 0.680281690140845
Validation ROC AUC: 0.7450704225352113


In [30]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss


def train_model(X_train, y_train):
    model = make_pipeline(
        SimpleImputer(strategy="median"),
        HistGradientBoostingClassifier(random_state=42)
    )
    model.fit(X_train, y_train)
    return model

def tune_model(X_train, y_train):
    param_grid = {
        "histgradientboostingclassifier__max_depth": [3, 5, 7],
        "histgradientboostingclassifier__learning_rate": [0.01, 0.05, 0.1],
        "histgradientboostingclassifier__max_iter": [100, 200, 500]
    }
    model = make_pipeline(
        SimpleImputer(strategy="median"),
        HistGradientBoostingClassifier(random_state=42)
    )
    grid = GridSearchCV(model, param_grid, scoring="roc_auc", cv=5, n_jobs=-1, verbose=3)
    grid.fit(X_train, y_train)
    return grid.best_estimator_, grid.best_params_


def evaluate(model, X_val, y_val):
    y_pred_proba = model.predict_proba(X_val)[:,1]
    auc = roc_auc_score(y_val, y_pred_proba)
    pr_auc = average_precision_score(y_val, y_pred_proba)
    brier = brier_score_loss(y_val, y_pred_proba)
    print(f"ROC AUC: {auc:.4f}, PR AUC: {pr_auc:.4f}, Brier: {brier:.4f}")
    return {"roc_auc": auc, "pr_auc": pr_auc, "brier": brier}


def cross_val_auc(model, X, y, cv=5):
    return cross_val_score(model, X, y, cv=cv, scoring="roc_auc").mean()

In [31]:
model, best_param = tune_model(X_train_moments, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


In [32]:
cross_val_auc(model, X_test_moments, y_test, cv=5)

np.float64(0.4531746031746032)

In [ ]:

# 2) CV + tuning, refit on all training
model, best_params, cv_results = train_cv_tuned_model(X_train_moments, y_train)


In [ ]:

# 3) Evaluate on your provided X_test / y_test
holdout_auc, proba_test = evaluate_on_holdout(model, X_test_moments, y_test)

# 4) Persist artifacts
joblib.dump(model, RESOURCES_DIR / "hgb_structbreak_model.joblib")
cv_results.to_csv(RESOURCES_DIR / "cv_results.csv", index=False)
proba_test.to_parquet(RESOURCES_DIR / "proba_test.parquet")

print("\nSaved:")
print("-", RESOURCES_DIR / "hgb_structbreak_model.joblib")
print("-", RESOURCES_DIR / "cv_results.csv")
print("-", RESOURCES_DIR / "proba_test.parquet")

In [37]:
# --- ROC AUC plot on the hold-out set ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Get ground truth (aligned to X_test's index if needed)
y_true = y_test.loc[X_test_moments.index] if isinstance(y_test, (pd.Series, pd.DataFrame)) else y_test
y_true = y_true.squeeze().astype(int).values  # ensure 1D int array

# Extract positive-class scores from your probabilities
# (works whether you kept numpy array from predict_proba or a DataFrame with "P(y=1)")
if isinstance(proba_test, np.ndarray):
    y_score = proba_test[:, 1]
else:
    y_score = pd.Series(proba_test).values  # adjust if you stored as a Series

# Compute ROC curve & AUC
fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_auc = auc(fpr, tpr)

# Optional: best threshold by Youden's J statistic
j_idx = np.argmax(tpr - fpr)
best_thr = thresholds[j_idx]

# Plot
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(fpr, tpr, lw=2, label=f"ROC AUC = {roc_auc:.4f}")
ax.plot([0, 1], [0, 1], linestyle="--", lw=1)
ax.scatter(fpr[j_idx], tpr[j_idx], s=30, label=f"Best thr ≈ {best_thr:.4f}")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve (hold-out)")
ax.legend(loc="lower right")
plt.tight_layout()
plt.show()

print(f"ROC AUC: {roc_auc:.6f}")
print(f"Best threshold (Youden J): {best_thr:.6f}")


ValueError: Data must be 1-dimensional, got ndarray of shape (101, 2) instead

In [ ]:
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier

def fit_ensemble_avg(X, y, seeds=(42, 123, 2025)):
    members = []
    for s in seeds:
        clf = HistGradientBoostingClassifier(
            random_state=s,
            max_iter=600, max_depth=5, learning_rate=0.06,
            min_samples_leaf=25, l2_regularization=1e-2, early_stopping=True
        )
        clf.fit(X, y)
        members.append(clf)
    return members

def predict_ensemble_avg(models, X):
    # average positive-class probability
    probs = [m.predict_proba(X)[:, 1] for m in models]
    return np.mean(probs, axis=0)

# Train
ens = fit_ensemble_avg(X_train_moments, y_train)
# Predict
proba_test_ens = predict_ensemble_avg(ens, X_test_moments)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

base_learners = [
    ("hgb1", HistGradientBoostingClassifier(random_state=42, max_iter=800, max_depth=5, learning_rate=0.06)),
    ("hgb2", HistGradientBoostingClassifier(random_state=123, max_iter=400, max_depth=7, learning_rate=0.03)),
]

stack = StackingClassifier(
    estimators=base_learners,
    final_estimator=LogisticRegression(max_iter=2000),
    stack_method="predict_proba",
    passthrough=False,  # set True if you want to pass original features to meta-learner
    n_jobs=-1
)

stack.fit(X_train_moments, y_train)
proba_test_stack = stack.predict_proba(X_test_moments)[:, 1]


In [ ]:
# pip install tensorflow
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models, optimizers

def build_sequence_arrays(X: pd.DataFrame, ids=None, max_len=None):
    """Return padded [num_ids, max_len, 2] (value, period), lengths, id_index."""
    if ids is None:
        ids = X.index.get_level_values("id").unique().tolist()
    seqs = []
    for i in ids:
        g = X.xs(i, level="id")
        seqs.append(np.stack([g["value"].values.astype("float32"),
                              g["period"].values.astype("float32")], axis=1))
    lengths = [len(s) for s in seqs]
    if max_len is None:
        max_len = max(lengths)
    # pad with zeros at the end
    seqs_pad = pad_sequences(seqs, maxlen=max_len, dtype="float32", padding="post", truncating="post")
    return np.asarray(seqs_pad), np.asarray(lengths), ids

def build_gru_model(input_len, input_dim=2, rnn_units=64, dropout=0.2):
    inp = layers.Input(shape=(input_len, input_dim))
    x = layers.Masking(mask_value=0.0)(inp)
    x = layers.Bidirectional(layers.GRU(rnn_units, return_sequences=False, dropout=dropout))(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    out = layers.Dense(1, activation="sigmoid")(x)
    model = models.Model(inp, out)
    model.compile(optimizer=optimizers.Adam(1e-3),
                  loss="binary_crossentropy", metrics=["AUC"])
    return model

# Build arrays
X_train_seq, train_len, train_ids = build_sequence_arrays(X_raw)
X_test_seq,  test_len,  test_ids  = build_sequence_arrays(X_test_raw, max_len=X_train_seq.shape[1])

# Targets aligned to train_ids
y_train_arr = y_train.loc[train_ids].astype(int).values

# Model
deep_model = build_gru_model(X_train_seq.shape[1], input_dim=2, rnn_units=64)
deep_model.fit(X_train_seq, y_train_arr, epochs=8, batch_size=64, validation_split=0.1, verbose=1)

# Probabilities
proba_test_deep = deep_model.predict(X_test_seq, verbose=0).squeeze()  # P(y=1)


In [ ]:
from sklearn.calibration import CalibratedClassifierCV

# e.g., calibrate the best HGB
best_hgb = HistGradientBoostingClassifier(
    random_state=42, max_iter=800, max_depth=5, learning_rate=0.06,
    min_samples_leaf=25, l2_regularization=1e-2
).fit(X_train_moments, y_train)

# method='isotonic' (non-parametric) or 'sigmoid' (Platt scaling)
cal = CalibratedClassifierCV(best_hgb, method="isotonic", cv=5)
cal.fit(X_train_moments, y_train)
proba_test_cal = cal.predict_proba(X_test_moments)[:, 1]
